# call the libraries

In [1]:
from pyspark.sql import SparkSession
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import io
import numpy as np
import os
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *
import scipy.stats as stats
import numpy as np
from sklearn.ensemble import IsolationForest
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pickle

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib.pyplot as plt
import numpy as np

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go



import numpy as np
import plotly
init_notebook_mode(connected=True)
%matplotlib inline



# initialize spark and spark context

In [2]:
spark = SparkSession.builder.appName('hospital-profile-sklearn-isolationforest-batch-score').getOrCreate()
sc = spark.sparkContext
sc.version

'2.1.1'

# lets load the Profile data for DOME9 with 3 attributes

In [3]:
tenant_name = "claim"
time_window="day"
entity_type="hospital"
anomaly_type="profile"
model_type="sklearn"
model_name="isolationforest"

BASE_PATH = "/Users/tuhinsharma/Documents/sstech/"+tenant_name
ANOMALY_DATA_REPOSITORY = BASE_PATH + "/models_data/data"

USER_PROFILE_DATA_PATH = ANOMALY_DATA_REPOSITORY + "/{entity_type}/{anomaly_type}/{time_window}.json"
data_path = USER_PROFILE_DATA_PATH.format(entity_type=entity_type,anomaly_type="profile",time_window=time_window)
    
ANOMALY_MODEL_REPOSITORY = BASE_PATH + "/models_data/model"
PROFILE_ANOMALY_MODEL_PATH = ANOMALY_MODEL_REPOSITORY + "/{entity_type}/{anomaly_type}/{time_window}/{model_type}/{model_name}"


# Query for Hospital Profile data for time interval "2019-06-21T12:41:20.000Z" to "2019-06-21T12:40:00.000Z" for batch scoring

In [4]:
query={
  "queryType": "topN",
  "dataSource": "hospitalprofile",
  "aggregations": [
    {
      "fieldName": "maximum_patient_age",
      "type": "longMax",
      "name": "MAX(maximum_patient_age)"
    },
    {
      "fieldName": "minimum_claim_amount",
      "fieldNames": [
        "minimum_claim_amount"
      ],
      "type": "doubleMin",
      "name": "MIN(minimum_claim_amount)"
    },
    {
      "fieldName": "minimum_patient_age",
      "fieldNames": [
        "minimum_patient_age"
      ],
      "type": "longMin",
      "name": "MIN(minimum_patient_age)"
    },
    {
      "fieldName": "total_claim_amount",
      "fieldNames": [
        "total_claim_amount"
      ],
      "type": "doubleSum",
      "name": "SUM(total_claim_amount)"
    },
    {
      "fieldName": "total_claim_count",
      "fieldNames": [
        "total_claim_count"
      ],
      "type": "longSum",
      "name": "SUM(total_claim_count)"
    },
    {
      "fieldName": "maximum_claim_amount",
      "fieldNames": [
        "maximum_claim_amount"
      ],
      "type": "doubleMax",
      "name": "MAX(maximum_claim_amount)"
    }
  ],
  "granularity": "all",
  "postAggregations": [],
  "intervals": "1901-01-01T00:00:00+00:00/2101-01-01T00:00:00+00:00",
  "filter": {
    "type": "and",
    "fields": [
      {
        "type": "bound",
        "dimension": "datetime",
        "lower": None,
        "lowerStrict": False,
        "upper": "2019-06-21T12:41:20.000Z",
        "upperStrict": False,
        "alphaNumeric": False
      },
      {
        "type": "bound",
        "dimension": "datetime",
        "lower": "2019-06-21T12:40:00.000Z",
        "lowerStrict": False,
        "upper": None,
        "upperStrict": False,
        "alphaNumeric": False
      }
    ]
  },
  "threshold": 100000,
  "metric": "MAX(maximum_patient_age)",
  "dimension": "hospital_id"
}

import requests
r = requests.post('http://0.0.0.0:8082/druid/v2/', json=query)
r.status_code
query_result = r.json()[0]["result"]

hospital_profile_df = pd.DataFrame.from_records(query_result)
hospital_profile_df.rename(columns={'MAX(maximum_claim_amount)': 'maximum_claim_amount',
                   'MAX(maximum_patient_age)': 'maximum_patient_age',
                   'MIN(minimum_claim_amount)': 'minimum_claim_amount',
                   'MIN(minimum_patient_age)': 'minimum_patient_age',
                   'SUM(total_claim_amount)': 'total_claim_amount',
                   'SUM(total_claim_count)': 'total_claim_count',
                  }, inplace=True)
hospital_profile_df.head(10)

,maximum_claim_amount,maximum_patient_age,minimum_claim_amount,minimum_patient_age,total_claim_amount,total_claim_count,hospital_id
0,46740.0,75,5710.0,23,120260.0,7,HOSP1
1,41970.0,75,6160.0,17,201470.0,9,HOSP12
2,44130.0,75,4580.0,15,265050.0,15,HOSP15
3,16460.0,75,1050.0,29,34140.0,4,HOSP16
4,17740.0,75,5730.0,20,69760.0,7,HOSP19
5,39440.0,75,1830.0,22,147930.0,10,HOSP22
6,39780.0,75,4780.0,15,144670.0,8,HOSP39
7,27890.0,75,2210.0,18,153050.0,12,HOSP42
8,10220.0,75,2870.0,30,22260.0,3,HOSP46
9,39630.0,75,7970.0,16,91800.0,4,HOSP6


In [5]:
model_path = PROFILE_ANOMALY_MODEL_PATH.format(entity_type=entity_type,anomaly_type=anomaly_type,time_window=time_window,\
                                 model_type=model_type,model_name=model_name)

### Load the Isolation Forest IF Pipelinemodel

In [6]:
model_dict = sc.pickleFile(model_path+"/if_pipeline_model").collect()[0]  
standard_scaler = model_dict["standard_scaler"]
one_hot_encoder = model_dict["one_hot_encoder"]
isolation_forest_model = model_dict["isolation_forest_model"]
cat_colnames = model_dict["cat_colnames"]
num_colnames = model_dict["num_colnames"]


### Load the Scoring Pipelinemodel

In [7]:
scoring_pipeline_model = sc.pickleFile(model_path+"/scoring_pipeline_model").collect()[0]  

In [8]:
if len(num_colnames)>0:
    num_data = hospital_profile_df[num_colnames].values.astype(np.float64)
    
if len(cat_colnames)>0:
    cat_data = hospital_profile_df[cat_colnames].values

if len(num_colnames)>0:
    num_data_normalized = standard_scaler.transform(num_data)
if len(cat_colnames)>0:
    cat_data_encoded = one_hot_encoder.transform(cat_data).toarray()

if len(num_colnames)>0 and len(cat_colnames)>0:
    data = np.concatenate((num_data_normalized,cat_data_encoded),axis=1)
elif len(cat_colnames):
    data = cat_data_encoded
elif len(num_colnames):
    data = num_data_normalized
    
score = isolation_forest_model.decision_function(data).reshape(-1, 1)*-1
pas = scoring_pipeline_model.transform(score)
hospital_profile_df["PAS"]=pas

In [9]:
result_score_sdf = spark.createDataFrame(hospital_profile_df)

## stats for PAS

In [10]:
result_score_sdf.select("PAS").describe().show()

+-------+------------------+
|summary|               PAS|
+-------+------------------+
|  count|               100|
|   mean|42.110565264665446|
| stddev| 22.83004630859148|
|    min|               0.0|
|    max|             100.0|
+-------+------------------+



# get binary response as normal/anomaly

In [11]:
def is_anomaly(value):
    if value > 90:
        return 1
    else:
        return 0
udf_is_anomaly = udf(is_anomaly,IntegerType())
result_score_sdf = result_score_sdf.withColumn("anomaly", udf_is_anomaly("PAS"))
result_score_df = result_score_sdf.toPandas()
result_score_df.head(20)

,maximum_claim_amount,maximum_patient_age,minimum_claim_amount,minimum_patient_age,total_claim_amount,total_claim_count,hospital_id,PAS,anomaly
0,46740.0,75,5710.0,23,120260.0,7,HOSP1,26.477735,0
1,41970.0,75,6160.0,17,201470.0,9,HOSP12,28.040234,0
2,44130.0,75,4580.0,15,265050.0,15,HOSP15,74.602982,0
3,16460.0,75,1050.0,29,34140.0,4,HOSP16,100.000000,1
4,17740.0,75,5730.0,20,69760.0,7,HOSP19,52.210127,0
5,39440.0,75,1830.0,22,147930.0,10,HOSP22,21.811993,0
6,39780.0,75,4780.0,15,144670.0,8,HOSP39,19.394919,0
7,27890.0,75,2210.0,18,153050.0,12,HOSP42,34.025264,0
8,10220.0,75,2870.0,30,22260.0,3,HOSP46,99.983930,1
9,39630.0,75,7970.0,16,91800.0,4,HOSP6,54.086945,0


# get the normal records

In [12]:
normal_sdf = result_score_sdf.where(result_score_sdf['anomaly'] == 0)
normal_sdf.select("anomaly").distinct().show()
print("normal record count {count}".format(count=normal_sdf.count()))
result_score_df[result_score_df.anomaly==0].head(10)

+-------+
|anomaly|
+-------+
|      0|
+-------+

normal record count 95


,maximum_claim_amount,maximum_patient_age,minimum_claim_amount,minimum_patient_age,total_claim_amount,total_claim_count,hospital_id,PAS,anomaly
0,46740.0,75,5710.0,23,120260.0,7,HOSP1,26.477735,0
1,41970.0,75,6160.0,17,201470.0,9,HOSP12,28.040234,0
2,44130.0,75,4580.0,15,265050.0,15,HOSP15,74.602982,0
4,17740.0,75,5730.0,20,69760.0,7,HOSP19,52.210127,0
5,39440.0,75,1830.0,22,147930.0,10,HOSP22,21.811993,0
6,39780.0,75,4780.0,15,144670.0,8,HOSP39,19.394919,0
7,27890.0,75,2210.0,18,153050.0,12,HOSP42,34.025264,0
9,39630.0,75,7970.0,16,91800.0,4,HOSP6,54.086945,0
10,19870.0,75,3380.0,24,53310.0,6,HOSP61,58.225121,0
11,48160.0,75,7340.0,21,187360.0,11,HOSP62,44.925989,0


# get the anomaly records

In [13]:
anomaly_sdf = result_score_sdf.where(result_score_sdf['anomaly'] == 1)
anomaly_sdf.select("anomaly").distinct().show()
print("anomaly record count {count}".format(count=anomaly_sdf.count()))
result_score_df[result_score_df.anomaly==1].head(10)

+-------+
|anomaly|
+-------+
|      1|
+-------+

anomaly record count 5


,maximum_claim_amount,maximum_patient_age,minimum_claim_amount,minimum_patient_age,total_claim_amount,total_claim_count,hospital_id,PAS,anomaly
3,16460.0,75,1050.0,29,34140.0,4,HOSP16,100.000000,1
8,10220.0,75,2870.0,30,22260.0,3,HOSP46,99.983930,1
15,18380.0,74,3360.0,35,33080.0,3,HOSP17,99.871176,1
75,31360.0,66,7250.0,42,82360.0,4,HOSP66,92.933407,1
76,24450.0,66,13690.0,21,56890.0,3,HOSP68,95.398532,1
